In [1]:
import pandas as pd
import numpy as np
import math
import urllib
import requests
import asyncio
from bs4 import BeautifulSoup as BS

In [60]:
#async functions

def snag_shit_pandas(url):
    response = pd.read_html(url)
    return response

async def snag_shit_soups(url):
    page = requests.get(url_list[j])
    soup = BS(page.content, 'lxml')
    response = soup.find_all('a') 
    return await response

In [ ]:
#asynchronous  info
# Get a reference to the event loop
loop = asyncio.get_event_loop()

# Create the list of shit we want to get asnyc
requests = [asyncio.ensure_future(snag_shit_pandas(someDumbassURL)),
            asyncio.ensure_future(snag_shit_pandas(someOtherDumbShitURL))]

# Run the requests
responses = loop.run_until_complete(asyncio.gather(*requests))

responses

#found this dope shit too. add it if ou want to quit long stuff
try:
    loop.run_forever()
except KeyboardInterrupt:
    # Canceling pending tasks and stopping the loop
    asyncio.gather(*asyncio.Task.all_tasks()).cancel()
    # Stopping the loop
    loop.stop()
    # Received Ctrl+C
    loop.close() 

In [2]:
#inputs stuff

tickers = ['MNTA', 'AON', 'OPHT', 'IIVI', 'MMM', 'GE']
filing_input_type = 'DEF+14'
table_var = "Summary Compensation Table"
period = 2

In [3]:
url_list = []

for i in range(0, len(tickers)):
    url_list.append('https://www.sec.gov/cgi-bin/browse-edgar?CIK=' + tickers[i] + '&owner=include&action=getcompany&type=' + filing_input_type)    
url_list

['https://www.sec.gov/cgi-bin/browse-edgar?CIK=MNTA&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=AON&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=OPHT&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=IIVI&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=MMM&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=GE&owner=include&action=getcompany&type=DEF+14']

In [4]:
#BS grabs links here
tag_list = []
document_links = []

for j in range(0,len(url_list)):
    page = requests.get(url_list[j])
    soup = BS(page.content, 'lxml')
    doc_link = soup.find_all('a')
    count = 0

    for i in range(0,len(doc_link)):
        x = doc_link[i].get('href')
        y = x.split('/')
        if len(y) > 1:
            if (y[1] == 'Archives' and count <= period):
                tag_list.append(x)
                count = count + 1

    for i in range(0,len(tag_list)):
        index_page = requests.get('https://www.sec.gov'+tag_list[i])
        soup2= BS(index_page.content, 'lxml')
        doc_link2 = soup2.find_all('a')
        document_links.append([tickers[j],doc_link2[8].get('href')])
    
document_links

[['MNTA',
  '/Archives/edgar/data/1235010/000104746918003290/a2235381zdef14a.htm'],
 ['MNTA',
  '/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm'],
 ['MNTA',
  '/Archives/edgar/data/1235010/000104746916012622/a2228452zdef14a.htm'],
 ['AON',
  '/Archives/edgar/data/1235010/000104746918003290/a2235381zdef14a.htm'],
 ['AON',
  '/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm'],
 ['AON',
  '/Archives/edgar/data/1235010/000104746916012622/a2228452zdef14a.htm'],
 ['AON', '/Archives/edgar/data/315293/000119312518138792/d471390ddef14a.htm'],
 ['AON', '/Archives/edgar/data/315293/000104746917003055/a2231270zdef14a.htm'],
 ['AON', '/Archives/edgar/data/315293/000104746916012756/a2228022zdef14a.htm'],
 ['OPHT',
  '/Archives/edgar/data/1235010/000104746918003290/a2235381zdef14a.htm'],
 ['OPHT',
  '/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm'],
 ['OPHT',
  '/Archives/edgar/data/1235010/000104746916012622/a2228452zd

In [5]:
filing_reader = []
for i in range(0,len(document_links)):
    url = 'https://www.sec.gov'+document_links[i][1]
    filing_reader.append([document_links[i][0],pd.read_html(url)])

In [46]:
#filing_reader[0][1][21][0].str.contains('Position' | 'salary').sum()
searchParameters = ("Name","Position","Salary","Stock","Award","Option","Non‑Equity","Incentive","Plan","Compensation","All Other","Total")
any(s in filing_reader[0][1][21][0] for s in searchParameters)

False

In [43]:
#need to fix

highestScore = 0
searchParameters = ("Name","Position","Salary","Stock","Award","Option","Non‑Equity","Incentive","Plan","Compensation","All Other","Total")
for filings in range(0, len(filing_reader)): #for all filings
    for i in range(0, len(filing_reader[filings][1])): #for each table
        frame_counter = 0
        for j in range(0,len(filing_reader[filings][1][i])): #for each column 
            if any(s in filing_reader[filings][1][i][j] for s in searchParameters):
                frame_counter += 1
            if (frame_counter > highestScore):
                bestmatch = filing_reader[filings][1][i][j]
                highestScore = frame_counter
        bestmatches.append([filing_reader[filings][0],bestmatch])
    
        

KeyError: 3

In [ ]:
bestmatches